In [1]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]


In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1156812477),
             ('parentid', 1156762483),
             ('user', 'David Gerard'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=5, tm_mday=24, tm_hour=19, tm_min=10, tm_sec=55, tm_wday=2, tm_yday=144, tm_isdst=-1)),
             ('comment', 'Generally Unreliable source, redundant')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

c:\Users\Raeed Zainuddin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Raeed Zainuddin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\debug\cli\debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\Raeed Zainuddin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\open_llama\modeling_open_llama.py:42: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  l

In [7]:
find_sentiment("I love you")

0.9998656511306763

In [8]:
find_sentiment("I hate you")

-0.9991129040718079

In [9]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Raeed Zainuddin\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Raeed Zainuddin\AppData\Local\Temp\ipykernel_17748\620939896.py", line 11, in <module>
    edits[date]["sentiments"].append(find_sentiment(comment))
                                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Raeed Zainuddin\AppData\Local\Temp\ipykernel_17748\3009866137.py", line 5, in find_sentiment
    sent = sentiment_pipeline([text[:250]])[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Raeed Zainuddin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\text_classification.py", line 155, in __call__
    result = super().__call__(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Raeed Zainuddin\AppData\Local\Programs\Python\Python311\Lib\site-packages

In [10]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
        
    del edits[key]["sentiments"]

In [11]:
edits

{'2009-03-08': {'edit_count': 4,
  'sentiment': -0.550525039434433,
  'neg_sentiment': 0.75},
 '2009-08-05': {'edit_count': 1,
  'sentiment': 0.7481209635734558,
  'neg_sentiment': 0.0},
 '2009-08-06': {'edit_count': 2,
  'sentiment': 0.9957457184791565,
  'neg_sentiment': 0.0},
 '2009-08-14': {'edit_count': 1,
  'sentiment': 0.9300208687782288,
  'neg_sentiment': 0.0},
 '2009-10-13': {'edit_count': 2,
  'sentiment': -0.2275010645389557,
  'neg_sentiment': 0.5},
 '2009-11-18': {'edit_count': 1,
  'sentiment': 0.883950412273407,
  'neg_sentiment': 0.0},
 '2009-12-08': {'edit_count': 1,
  'sentiment': -0.9869275689125061,
  'neg_sentiment': 1.0},
 '2009-12-17': {'edit_count': 1,
  'sentiment': -0.9975171089172363,
  'neg_sentiment': 1.0},
 '2010-02-23': {'edit_count': 1,
  'sentiment': -0.9994946718215942,
  'neg_sentiment': 1.0},
 '2010-03-18': {'edit_count': 1,
  'sentiment': 0.8758774995803833,
  'neg_sentiment': 0.0},
 '2010-04-13': {'edit_count': 4,
  'sentiment': 0.8443553596735001

In [12]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [13]:
edits_df


,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-08-05,1,0.748121,0.00
2009-08-06,2,0.995746,0.00
2009-08-14,1,0.930021,0.00
2009-10-13,2,-0.227501,0.50
...,...,...,...
2011-05-08,2,-0.961282,1.00
2011-05-09,2,0.839071,0.00
2011-05-10,1,0.874822,0.00
2011-05-11,1,0.874822,0.00


In [14]:
edits_df.index = pd.to_datetime(edits_df.index)

In [15]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [16]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2023-05-20', '2023-05-21', '2023-05-22', '2023-05-23',
               '2023-05-24', '2023-05-25', '2023-05-26', '2023-05-27',
               '2023-05-28', '2023-05-29'],
              dtype='datetime64[ns]', length=5196, freq='D')

In [17]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [18]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2023-05-25,0,0.000000,0.00
2023-05-26,0,0.000000,0.00
2023-05-27,0,0.000000,0.00
2023-05-28,0,0.000000,0.00


In [19]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [20]:
rolling_edits = rolling_edits.dropna()

In [21]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025
2009-04-07,0.000000,0.000000,0.000
2009-04-08,0.000000,0.000000,0.000
2009-04-09,0.000000,0.000000,0.000
2009-04-10,0.000000,0.000000,0.000
...,...,...,...
2023-05-25,0.000000,0.000000,0.000
2023-05-26,0.000000,0.000000,0.000
2023-05-27,0.000000,0.000000,0.000
2023-05-28,0.000000,0.000000,0.000


In [22]:
rolling_edits.to_csv("wikipedia_edits.csv")